# Ingest massive amounts of data to a Vector DB (Amazon OpenSearch)
**_Use of Amazon OpenSearch as a vector database for storing embeddings_**

This notebook works well with the `Data Science 2.0` kernel on a SageMaker Studio `ml.t3.medium` instance.

Here is a list of packages that are used in this notebook.
```
!pip freeze | grep -E "sagemaker|boto3|haystack|opensearch|transformers|torch"
----------------------------------------------------------------------------------------
boto3 @ file:///home/conda/feedstock_root/build_artifacts/boto3_1683763173043/work
farm-haystack==1.21.0
opensearch-py==2.3.1
sagemaker==2.188.0
sagemaker-experiments==0.1.43
sagemaker-pytorch-training==2.8.0
sagemaker-training==4.5.0
sentence-transformers==2.2.2
smdebug @ file:///tmp/sagemaker-debugger
torch==2.0.0
torchaudio==2.0.1
torchdata @ file:///opt/conda/conda-bld/torchdata_1679615656247/work
torchtext==0.15.1
torchvision==0.15.1
transformers==4.32.1
```

## Step 1: Setup
Install the required packages.

In [ ]:
!pip install -U -r requirements.txt --quiet
!pip install -U sagemaker --quiet
!pip install -U sagemaker-studio-image-build==0.6.0 --quiet

In [ ]:
!pip freeze | grep -E "sagemaker|boto3|haystack|opensearch|transformers|torch"

## Step 2: Download the data 

In this step we use `wget` to crawl a Python documentation style website data. All files other than `html`, `txt` and `md` are removed. **This data download would take a few minutes**.

In [ ]:
%%sh

mkdir -p data
cd ./data
wget https://raw.githubusercontent.com/deepset-ai/haystack-sagemaker/main/data/opensearch-documentation-2.7.json
wget https://raw.githubusercontent.com/deepset-ai/haystack-sagemaker/main/data/opensearch-website.json

In [ ]:
import boto3
import sagemaker

sagemaker_session = sagemaker.session.Session()
aws_region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()

## Step 3: Load data into OpenSearch

We now have a working script that is able to ingest data into an OpenSearch index. But for this to work for massive amounts of data we need to scale up the processing by running this code in a distributed fashion. We will do this using Sagemkaer Processing Job. This involves the following steps:

1. Create a custom container in which we will install the `langchain` and `opensearch-py` packges and then upload this container image to Amazon Elastic Container Registry (ECR).
2. Use the Sagemaker `ScriptProcessor` class to create a Sagemaker Processing job that will run on multiple nodes.
    - The data files available in S3 are automatically distributed across in the Sagemaker Processing Job instances by setting `s3_data_distribution_type='ShardedByS3Key'` as part of the `ProcessingInput` provided to the processing job.
    - Each node processes a subset of the files and this brings down the overall time required to ingest the data into Opensearch.
    - Each node also uses Python `multiprocessing` to internally also parallelize the file processing. Thus, **there are two levels of parallelization happening, one at the cluster level where individual nodes are distributing the work (files) amongst themselves and another at the node level where the files in a node are also split between multiple processes running on the node**.

### Create custom container

We will now create a container locally and push the container image to ECR. **The container creation process takes about 1 minute**.

In [ ]:
DOCKER_IMAGE = "haystack-opensearch-indexing-pipeline"
DOCKER_IMAGE_TAG = "latest"

In [ ]:
!cd ./container && sm-docker build . --repository {DOCKER_IMAGE}:{DOCKER_IMAGE_TAG}

### Create and run the Sagemaker Processing Job

Now we will run the Sagemaker Processing Job to ingest the data into OpenSearch.

In [ ]:
import sys
import time
import logging

from typing import List

logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s,%(module)s,%(processName)s,%(levelname)s,%(message)s', level=logging.INFO, stream=sys.stderr)

In [ ]:

import json
import boto3


def get_opensearch_endpoint(stack_name: str, region_name: str = 'us-east-1'):
    cf_client = boto3.client('cloudformation', region_name=region_name)
    response = cf_client.describe_stacks(StackName=stack_name)
    outputs = response["Stacks"][0]["Outputs"]

    ops_endpoint = [e for e in outputs if e['ExportName'] == 'OpenSearchDomainEndpoint'][0]
    ops_endpoint_name = ops_endpoint['OutputValue']
    return ops_endpoint_name


def get_secret_name(stack_name: str, region_name: str = 'us-east-1'):
    cf_client = boto3.client('cloudformation', region_name=region_name)
    response = cf_client.describe_stacks(StackName=stack_name)
    outputs = response["Stacks"][0]["Outputs"]

    secrets = [e for e in outputs if e['ExportName'] == 'MasterUserSecretId'][0]
    secret_name = secrets['OutputValue']
    return secret_name


def get_secret(secret_name: str, region_name: str = 'us-east-1'):
    client = boto3.client('secretsmanager', region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    secret = get_secret_value_response['SecretString']

    return json.loads(secret)

In [ ]:
CFN_STACK_NAME = "RAGHaystackOpenSearchStack"

opensearch_domain_endpoint = get_opensearch_endpoint(CFN_STACK_NAME, region_name=aws_region)
opensearch_secretid = get_secret_name(CFN_STACK_NAME, region_name=aws_region)

In [ ]:
app_name = "haystack-rag-app"
account_id = boto3.client("sts").get_caller_identity()["Account"]
aws_role = sagemaker.get_execution_role()

In [ ]:
from sagemaker.processing import (
    ProcessingInput,
    ScriptProcessor
)

# setup the parameters for the job
base_job_name = f"{app_name}-job"
tags = [{"Key": "data", "Value": app_name}]

# use the custom container we just created
image_uri = f"{account_id}.dkr.ecr.{aws_region}.amazonaws.com/{DOCKER_IMAGE}:{DOCKER_IMAGE_TAG}"

# instance type and count determined via trial and error: how much overall processing time
# and what compute cost works best for your use-case
instance_type = "ml.c5.2xlarge"
instance_count = 1
logger.info(f"base_job_name={base_job_name}, tags={tags}, image_uri={image_uri}, instance_type={instance_type}, instance_count={instance_count}")

# setup the ScriptProcessor with the above parameters
processor = ScriptProcessor(base_job_name=base_job_name,
                            image_uri=image_uri,
                            role=aws_role,
                            instance_type=instance_type,
                            instance_count=instance_count,
                            command=["python3"],
                            tags=tags)

# setup input from S3, note the ShardedByS3Key, this ensures that 
# each instance gets a random and equal subset of the files in S3.
inputs = [ProcessingInput(source="./data",
                          destination='/opt/ml/processing/input',
                          s3_data_distribution_type='ShardedByS3Key',
                          s3_data_type='S3Prefix')]


logger.info(f"creating an opensearch index with name=document")

# ready to run the processing job
st = time.time()
processor.run(code="container/load_data_into_opensearch.py",
              inputs=inputs,
              outputs=[],
              arguments=["--opensearch-endpoint", opensearch_domain_endpoint,
                         "--opensearch-secret-id", opensearch_secretid,
                         "--aws-region", aws_region,
                         "--input-data-dir", "/opt/ml/processing/input"
])

time_taken = time.time() - st
logger.info(f"processing job completed, total time taken={time_taken}s")

preprocessing_job_description = processor.jobs[-1].describe()
logger.info(preprocessing_job_description)

## Step 4: Do a similarity search for for user input to documents (embeddings) in OpenSearch

## Cleanup

To avoid incurring future charges, delete the resources. You can do this by deleting the CloudFormation template used to create the IAM role and SageMaker notebook.

---

## Conclusion
In this notebook we were able to see how to use LLMs deployed on a SageMaker Endpoint to generate embeddings and then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. We used langchain as an abstraction layer to talk to both the SageMaker Endpoint as well as OpenSearch.

---

## References

  * [Build a powerful question answering bot with Amazon SageMaker, Amazon OpenSearch Service, Streamlit, and LangChain](https://aws.amazon.com/blogs/machine-learning/build-a-powerful-question-answering-bot-with-amazon-sagemaker-amazon-opensearch-service-streamlit-and-langchain/)
  * [Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/)
  * [LangChain](https://python.langchain.com/docs/get_started/introduction.html) - A framework for developing applications powered by language models.